# Transformer

- Encoder: O Encoder recebe a sequência de entrada (por exemplo, uma frase em português) e a processa para criar uma representação vetorial de alta qualidade dessa sequência. Essa representação captura o significado e o contexto de cada palavra em relação às outras palavras da frase.
- Decoder: O Decoder recebe a representação gerada pelo Encoder e a utiliza para gerar uma sequência de saída (por exemplo, a tradução da frase para o inglês).


# Masked Language Model

Um Masked Language Model (MLM) é um tipo de modelo de linguagem amplamente utilizado em processamento de linguagem natural.

Durante o treinamento, uma parte dos tokens (palavras ou subpalavras) no texto de entrada é substituída por um token especial de máscara, como "[MASK]". O objetivo do modelo é prever corretamente quais eram os tokens originais que foram mascarados.

Essa estratégia obriga o modelo a aprender contextos ricos e relações entre as palavras, o que é fundamental para o desempenho em diversas tarefas, como análise de sentimentos, tradução, e resposta a perguntas. Modelos famosos que utilizam essa técnica incluem o BERT, que demonstrou ganhos significativos em várias benchmarks de NLP .


## IMBD

Para este projeto, será utilizado a base do IMBD.

## Configuração

In [1]:
import os

os.environ[ "KERAS_BACKEND" ] = "torch"  # or jax, or tensorflow

import keras_hub

import keras
from keras import layers
from keras.layers import TextVectorization

from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint

In [2]:
@dataclass
class Config:
    MAX_LEN = 256
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128
    NUM_HEAD = 8  # used in bert model
    FF_DIM = 128  # used in bert model
    NUM_LAYERS = 1


config = Config()

# Carregando os dados

Primeiro, vamos carregar os dados que estão na pasta "aclImbd".

Duas funções serão utilizadas para isso:

- Uma irá criar uma lista contendo o conteúdo dos arquivos.
- A outra ficará responsável por criar um dataframe.

In [3]:
def get_text_list_from_files( files ) -> list[ str ]:
    """
       Esta função irá retornar uma lista contendo todas as frases dos arquivos.
    """
    text_list: list[ str ] = [ ]
    for name in files:
        with open( name, "r", encoding = "utf-8" ) as f:
            for line in f:
                text_list.append( line )
    return text_list


def get_data_from_text_files( folder_name ):
    # Arquivos de texto com avaliações positivas e negativas
    pos_files = glob.glob( f"aclImdb/{folder_name}/pos/*.txt" )
    neg_files = glob.glob( f"aclImdb/{folder_name}/neg/*.txt" )

    # Listas com as avaliações
    pos_texts: list[ str ] = get_text_list_from_files( pos_files )
    neg_texts: list[ str ] = get_text_list_from_files( neg_files )

    # Criação de um dataframe, com colunas chamadas "review" e "sentiment"
    df = pd.DataFrame(
            {
                # Concatenação das listas
                "review": pos_texts + neg_texts,
                # Criação de uma nova lista
                "sentiment": [ 0 ] * len( pos_texts ) + [ 1 ] * len( neg_texts ),
            }
    )

    # Sample -> pega uma amostra aleatória
    # len(df) -> do tamanho do df original
    # reset_index -> ao usar sample, o índice original das linhas é mantido
    df = df.sample( len( df ) ).reset_index( drop = True )
    return df


train_df = get_data_from_text_files( "train" )
test_df = get_data_from_text_files( "test" )

# Concatenação dos dataframes para realizar pré-processamento em toda a base
all_data = pd.concat( [ train_df, test_df ], ignore_index = True )

In [4]:
train_df.head()

,review,sentiment
0,"this is a terrible, terrible film!!!!!!!!!<br ...",1
1,The wonderfully urbane Ronald Coleman is show-...,0
2,You spend most of this two-hour film wondering...,0
3,The film as entertainment is very good and Jim...,0
4,"In 1988, Paperhouse was hailed as a ""thinking ...",1


In [5]:
import tensorflow as tf


def custom_standardization( input_data ):
    """Normalização de texto. """
    # Converter todas as letras para minúsculas
    lowercase = tf.strings.lower( input_data )

    # Expressão Regular para remover a tag HTML
    stripped_html = tf.strings.regex_replace( lowercase, "<br />", " " )

    # Remover caracteres especiais
    return tf.strings.regex_replace(
            stripped_html,
            "[%s]" % re.escape( "!#$%&'()*+,-./:;<=>?@\^_`{|}~" ),
            ""
    )

## Vetorização de Texto

Para um transformer, a vetorização de um texto é o processo fundamental de transformar o texto bruto em uma representação numérica que o modelo possa entender e processar. Em essência, é como traduzir a linguagem humana para a linguagem matemática que o transformer consegue trabalhar.

Imagine que o transformer é um computador que só entende números. Para que ele consiga ler e compreender um texto, precisamos converter cada palavra (ou parte da palavra) em um conjunto de números. Esse conjunto de números é o que chamamos de vetor.

Aqui está um detalhamento do processo de vetorização para um transformer:

1. Tokenização: O primeiro passo é dividir o texto em unidades menores, chamadas tokens. Um token pode ser uma palavra inteira, parte de uma palavra (subpalavra), ou até mesmo um caractere. Por exemplo, a frase "O gato comeu o rato" poderia ser tokenizada como: ["O", "gato", "comeu", "o", "rato"].

2. Criação do Vocabulário: Em seguida, é criado um vocabulário, que é uma lista de todos os tokens únicos presentes no conjunto de dados de treinamento do modelo. Cada token nesse vocabulário recebe um índice único.

3. Indexação: Cada token no texto de entrada é então mapeado para o seu índice correspondente no vocabulário. Usando o exemplo anterior e supondo um vocabulário, os tokens poderiam ser convertidos em índices como: [10, 25, 50, 10, 75].

4. Embedding: A etapa crucial para transformers é a criação de embeddings. Em vez de simplesmente usar os índices brutos, cada índice é transformado em um vetor denso de números reais. Esse vetor captura o significado semântico e as relações entre as palavras.

    - Word Embeddings: Cada palavra (ou token) é associada a um vetor de baixa dimensionalidade (por exemplo, 512 ou 768 dimensões). Palavras com significados semelhantes tendem a ter vetores próximos no espaço vetorial. Por exemplo, os vetores para "gato" e "felino" provavelmente estarão mais próximos do que os vetores para "gato" e "carro".

    - Positional Embeddings: Transformers também precisam entender a ordem das palavras em uma frase. Para isso, são adicionados embeddings posicionais aos word embeddings. Esses vetores codificam a posição de cada token na sequência, permitindo que o modelo diferencie entre "o gato comeu o rato" e "o rato comeu o gato".

5. Input para o Transformer: Os vetores resultantes (a soma dos word embeddings e positional embeddings para cada token) são então alimentados como entrada para as diferentes camadas do transformer (como as camadas de atenção).

In [6]:
def get_vectorize_layer( texts: list[ str ], vocab_size: int, max_seq: int, special_tokens: list = [ "[MASK]" ] ):
    """Build Text vectorization layer

    Args:
      texts (list): List of string i.e input texts
      vocab_size (int): vocab size
      max_seq (int): Maximum sequence length.
      special_tokens (list, optional): List of special tokens. Defaults to ['[MASK]'].

    Returns:
        layers.Layer: Return TextVectorization Keras Layer
    """

    # Criação da camada de TextVectorization
    vectorize_layer = TextVectorization(
            max_tokens = vocab_size,  # Define o tamanho máximo do vocabulário
            output_mode = "int",  # Define que a saída deve ser uma sequência de números inteiros
            standardize = custom_standardization,  # Aplicar função de pré-processamento
            output_sequence_length = max_seq,  # Garantir que toda sequência de saída tenha comprimento max_seq
    )

    # todo Mostrar um exemplo

    # Adaptação aos textos de entrada
    # A camada "sabe" como mapear palavras para números inteiros, com base nas entradas
    vectorize_layer.adapt( texts )

    # Obtém o vocabulário aprendido
    vocab = vectorize_layer.get_vocabulary()

    # Por padrão, o TextVectorization coloca "" (para padding, índice 0) e "[UNK]" (para palavras desconhecidas,
    # índice 1) no início do vocabulário.

    # Pegando uma porção do vocabulário:
    # - Ignorando "" e "[UNK]"
    # - Pega quase tudo, deixando espaço suficiente para special_tokens
    vocab = vocab[ 2: vocab_size - len( special_tokens ) ] + special_tokens

    # Atualiza o vocabulário da camada
    vectorize_layer.set_vocabulary( vocab )

    # Retorna a camada
    return vectorize_layer

In [7]:
# Pegando a camada de vetorização
vectorize_layer = get_vectorize_layer(
        all_data.review.values.tolist(),
        config.VOCAB_SIZE,
        config.MAX_LEN,
        special_tokens = [ "[mask]" ],
)

# Processamento de um novo dado: "[mask]"
# - Aplica a função de pré-processamento
# - Divide em token, pega seu id e cria uma sequência de comprimento config.MAX_LEN
# - Converte o resultado para um array NumPy
# - Pega o id do "[mask]"
mask_token_id = vectorize_layer( [ "[mask]" ] ).cpu().numpy()[ 0 ][ 0 ]

In [8]:
def encode( texts ):
    """ Retorna um array NumPy das sequências numéricas dos textos de entrada."""
    # Criação das sequências numéricas para os textos de entrada
    encoded_texts = vectorize_layer( texts )
    # Retorna as sequências como um array NumPy
    return encoded_texts.cpu().numpy()

### Contexto para o código: Mascaramento

Em essência, o mascaramento envolve ocultar aleatoriamente algumas das palavras (ou tokens) em uma sequência de texto de entrada. O objetivo é fazer com que o modelo aprenda a prever as palavras que foram mascaradas, com base no contexto das palavras vizinhas não mascaradas.

Imagine a frase: "O gato está dormindo no tapete."

No processo de mascaramento, poderíamos aleatoriamente escolher algumas palavras para ocultar, substituindo-as por um token especial, geralmente chamado [MASK]. Por exemplo, a frase poderia se tornar:

"O [MASK] está dormindo no [MASK]."

O modelo de linguagem, durante o treinamento, receberia essa versão mascarada da frase como entrada e teria como objetivo prever as palavras originais que foram substituídas por [MASK]. Neste caso, o modelo deveria aprender a prever "gato" para o primeiro [MASK] e "tapete" para o segundo.

In [9]:
def get_masked_input_and_labels( encoded_texts ):
    # Cria um array NumPy com o mesmo tamanho de encoded_texts preenchido com números aleatórios entre 0 e 1
    # Vai comparar cada um dos números com 0.15, se for maior ou igual, será True, do contrário, será False
    # inp_mask será um array de booleanos de mesmo tamanho que encoded_texts
    inp_mask = np.random.rand( *encoded_texts.shape ) < 0.15

    # Não deixa realizar o mascaramento em tokens especiais
    inp_mask[ encoded_texts <= 2 ] = False

    # Cria um array com o mesmo tamanho de encoded_texts preenchido com o valor -1
    # O valor -1 é usado para indicar que esses tokens não são alvos para a previsão durante o treinamento
    labels = -1 * np.ones( encoded_texts.shape, dtype = int )

    # Para as posições no array qye são True em inp_mask, os valores dos IDs dos tokens são atribuídos a labels
    # Assim, labels terá os IDs dos tokens que foram mascarados, e o modelo terá que prever esses IDs
    labels[ inp_mask ] = encoded_texts[ inp_mask ]

    # Cria uma cópia de encoded_texts
    encoded_texts_masked = np.copy( encoded_texts )

    # Cria uma nova máscara booleana
    # Ela é True apenas nas posições onde inp_mask também é True E um novo número aleatório gerado para
    # essa posição é menor que 0.9
    # Apenas 90% dos 15% dos tokens selecionados para mascaramente, serão, de fato, mascarados
    inp_mask_2mask = inp_mask & (np.random.rand( *encoded_texts.shape ) < 0.90)

    # Atualizando posições onde inp_mask_2mask são True para a máscara
    encoded_texts_masked[ inp_mask_2mask ] = mask_token_id

    # Cria uma nova máscara booleana
    # Ela é True apenas nas posições onde inp_mask_2mask também é True E um novo número aleatório gerado para
    # essa posição é menor que 1/9
    # 1/9 dos 90% serão tokens aleatórios
    inp_mask_2random = inp_mask_2mask & (np.random.rand( *encoded_texts.shape ) < 1 / 9)

    # Nas posições onde inp_mask_2random é True, o token em encoded_texts_masked será um token aleatório
    # Gera um array de números aleatórios partindo de 3 e indo até antes de mask_token_id
    # Começou em 3 porque as primeiras posições foram excluídas
    encoded_texts_masked[ inp_mask_2random ] = np.random.randint(
            3, mask_token_id, inp_mask_2random.sum()
    )

    # Cria um array de mesmo tamanho que labels preenchido com 1
    sample_weights = np.ones( labels.shape )

    # Nas posições dos tokens que têm -1, o valor em sample_weights será 0
    # Isso significa que a perda durante o treinamento será calculada apenas para os tokens
    # que foram realmente mascarados
    sample_weights[ labels == -1 ] = 0

    # Cria uma cópia de encoded_texts
    y_labels = np.copy( encoded_texts )

    # A versão da entrada com alguns tokens substituídos por [MASK] ou por tokens aleatórios
    # Os tokens originais da entrada, que servem como os rótulos para o treinamento
    # Um array de pesos que indica quais posições em y_labels devem ser consideradas no cálculo da perda
    return encoded_texts_masked, y_labels, sample_weights

## Codificação e Datasets

1. Classificação de Sentimentos: Cria datasets de treinamento e teste (train_classifier_ds e test_classifier_ds) onde as revisões são codificadas e emparelhadas com seus rótulos de sentimento.

2. Modelo de Linguagem Mascarada: Cria um dataset (mlm_ds) para treinar um modelo a prever palavras mascaradas em um conjunto de todas as revisões.

In [10]:
# Codificação das reviews do dataframe de treinamento
x_train = encode( train_df.review.values )

# Pegando os sentimentos de cada review
y_train = train_df.sentiment.values

train_classifier_ds = (
    # Cria um dataset onde cada elemento é uma tupla: a review codificada e o seu sentimento
    tf.data.Dataset.from_tensor_slices( (x_train, y_train) )
    # Embaralha os dados do dataset
    .shuffle( 1000 )
    # Agrupa os dados em lotes
    .batch( config.BATCH_SIZE )
)

In [11]:
# Codificação das reviews do dataframe de teste
x_test = encode( test_df.review.values )

# Pegando os sentimentos de cada review
y_test = test_df.sentiment.values

test_classifier_ds = (
    # Cria um dataset onde cada elemento é uma tupla: a review codificada e o seu sentimento
    tf.data.Dataset.from_tensor_slices( (x_test, y_test) )
    # Agrupa os dados em lotes
    .batch( config.BATCH_SIZE )
)

# Armazenando o dataframe de teste para um uso futuro
test_raw_classifier_ds = test_df

In [12]:
# Aplica a codificação em toda a base
x_all_review = encode( all_data.review.values )

# Passa a codificação da base para realizar o mascaramento dos dados

# x_masked_train: As revisões com alguns tokens mascarados (substituídos por um token especial ou por tokens aleatórios).
#   Esta será a entrada para o modelo de linguagem mascarada.

# y_masked_labels: Os rótulos para a tarefa de mascaramento.

# sample_weights: Pesos que indicam quais tokens devem ser considerados no cálculo da perda
x_masked_train, y_masked_labels, sample_weights = get_masked_input_and_labels(
        x_all_review
)

# Cria um dataset onde, cada elemento será uma tupla:
# - As reviews mascaradas
# - Rótulos originais
# - Pesos de amostra
mlm_ds = tf.data.Dataset.from_tensor_slices(
        (x_masked_train, y_masked_labels, sample_weights)
)

# Embaralha o dataset e agrupa em lotes
mlm_ds = mlm_ds.shuffle( 1000 ).batch( config.BATCH_SIZE )

## Criação do modelo BERT (Bidirectional Encoder Representations from Transformers)

1. Bloco transformer com componentes/operacoes Multi-Head Self-Attention, Feed Foward Network e dropout.

2. Customização do modelo, definindo como calcular as metricas que serao utilizadas, neste caso calculando a perda com Sparse Categorical Crossentropy e atualizando com a média da perda.

3. Construção do modelo BERT para o mlm.

4. Callback para a geração de texto para tokens mascarados e apresentação de resultados.

In [13]:
def bert_module(query, key, value, i):
    
    #Fazer uma "self-attention" com multiplas cabeças
    #self attention e uma maneira do modelo enteder as relacoes de uma sequencia de palavras
    #por ter mais de 1 cabeça permite notar essas relacoes em diferentes partes do texto
    attention_output = layers.MultiHeadAttention(
        num_heads=config.NUM_HEAD,
        key_dim=config.EMBED_DIM // config.NUM_HEAD,
        name="encoder_{}_multiheadattention".format(i),
    )(query, key, value)
    
    #Dropout para ajudar na regularizacao (evitar overfitting)
    attention_output = layers.Dropout(0.1, name="encoder_{}_att_dropout".format(i))(
        attention_output
    )
    
    #Layer normalization da atencao que foi calculada e da entrada original
    #serve para garantir o processo de informacao de maneira consistente, ajudando na performance e eficiencia
    attention_output = layers.LayerNormalization(
        epsilon=1e-6, name="encoder_{}_att_layernormalization".format(i)
    )(query + attention_output)

    # Feed-forward layer, serve para que informacoes em estagios tadios do processamento sejam
    #enviados para estagios mais iniciais
    ffn = keras.Sequential(
        [
            layers.Dense(config.FF_DIM, activation="relu"),#expande  dimensao para FF_DIMe ativacao ReLU
            layers.Dense(config.EMBED_DIM),#volta para a dimensao original
        ],
        name="encoder_{}_ffn".format(i),
    )
    
    #aplicacao de dropout novamente
    ffn_output = ffn(attention_output)
    ffn_output = layers.Dropout(0.1, name="encoder_{}_ffn_dropout".format(i))(
        ffn_output
    )
    
    #layer normalization novamente com a saida da camada de atencao
    sequence_output = layers.LayerNormalization(
        epsilon=1e-6, name="encoder_{}_ffn_layernormalization".format(i)
    )(attention_output + ffn_output)
    
    #o resultado possui a mesma dimensao da entrada porem com representacoes melhores
    #mesmos dados porem enriquecidos de contexto
    return sequence_output


In [14]:
#Funcao de perda de entropia cruzada esparsa
#usada quando tem duas ou mais classes de label
loss_fn = keras.losses.SparseCategoricalCrossentropy(reduction=None)
#media dos valores de perda ao longo do treinamento
loss_tracker = keras.metrics.Mean(name="loss")

#esta classe serve para calcular as metricas e a perda de maneira customizada
#assim temos um controle maior do treinamento
class MaskedLanguageModel(keras.Model):

    def compute_loss(self, x=None, y=None, y_pred=None, sample_weight=None):
        #ja que na funcao de perda nao tem reducao vamos ter um valor de perda pra cada exemplo
        #entao o fazemos com que a soma total seja o que sera otimizada
        loss = loss_fn(y, y_pred, sample_weight)
        loss_tracker.update_state(loss, sample_weight=sample_weight)
        return keras.ops.sum(loss)

    def compute_metrics(self, x, y, y_pred, sample_weight):
        # Retorna um dicionario com a perda media do loss_tracker
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        #este serve para que nao seja necessario o reset manual das metricas
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [loss_tracker]

In [ ]:
def create_masked_language_bert_model():
    #configura a camada de entrada para que seja uma lista de tokens INT
    inputs = layers.Input((config.MAX_LEN,), dtype="int64")
    
    #converte os IDs dos tokens em vetores
    #cava vetor tera  dimensao de EMBED_DIM
    word_embeddings = layers.Embedding(
        config.VOCAB_SIZE, config.EMBED_DIM, name="word_embedding"
    )(inputs)
    
    #cria a informacao adicional das posicoes dos tokens para o modelo usando a bibliote do keras_hub
    position_embeddings = keras_hub.layers.PositionEmbedding(
        sequence_length=config.MAX_LEN
    )(word_embeddings)
    
    #combina as duas informacoes criadas, a de semantica de a de posicao
    embeddings = word_embeddings + position_embeddings

    encoder_output = embeddings
    
    #para cada camada se aplica o bert, um bloco transformer contendo Multi-Head Self-Attention e Feed-Forward Network
    for i in range(config.NUM_LAYERS):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, i)

    #camada de classificacao para o mlm, transformando a saída do encoder em predições de tokens
    #a função de ativação softmax transforma as saídas brutas da rede neural em um vetor de probabilidades
    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(
        encoder_output
    )
    
    #criar a instancia da classe mlm que ja definimos
    mlm_model = MaskedLanguageModel(inputs, mlm_output, name="masked_bert_model")

    #otimizacao usando "Adam" e compilacao
    optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    mlm_model.compile(optimizer=optimizer)
    return mlm_model

#mapeamento de ID para token e vice versa para facilitar interpretacao dos dados
id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y: x for x, y in id2token.items()}

In [ ]:
#por herdar keras.callbacks.Callback pode ser usada durante o treinamento
class MaskedTextGenerator(keras.callbacks.Callback):
    def __init__(self, sample_tokens, top_k=5):
        self.sample_tokens = sample_tokens #exemplo de entrada
        self.k = top_k #quantos candidatos serao considerados para cada mascara

    #sequencia de tokens para uma string
    def decode(self, tokens):
        return " ".join([id2token[t] for t in tokens if t != 0])

    #converte o seu ID para seu token correspondente
    def convert_ids_to_tokens(self, id):
        return id2token[id]

    #executado ao final de cada epoca
    def on_epoch_end(self, epoch, logs=None):
        #recebe as previsoes do modelo para o exemplo fornecido
        prediction = self.model.predict(self.sample_tokens)

        #procura onde esta os tokens de mascara
        masked_index = np.where(self.sample_tokens == mask_token_id)
        masked_index = masked_index[1]
        
        #pega as probabilidades de predicao para os tokens mascarados
        mask_prediction = prediction[0][masked_index]

        #ordena e armazena em values os top_k (5 neste caso) maiores probabilidades
        top_indices = mask_prediction[0].argsort()[-self.k :][::-1]
        values = mask_prediction[0][top_indices]

        #para cada entre os top candidatos cria uma copia e substitui
        #depois printa o texto original e os resultados com as palavras preditas junto com a probabilidade e token
        for i in range(len(top_indices)):
            p = top_indices[i]
            v = values[i]
            tokens = np.copy(sample_tokens[0])
            tokens[masked_index[0]] = p
            result = {
                "input_text": self.decode(sample_tokens[0].numpy()),
                "prediction": self.decode(tokens),
                "probability": v,
                "predicted mask token": self.convert_ids_to_tokens(p),
            }
            pprint(result)

#converte o exemplo para uma sequencia de IDs e instacia com o callback com o exemplo, este sera usado nas predicoes no final de cada epoca
sample_tokens = vectorize_layer(["I have watched this [mask] and it was awesome"])
generator_callback = MaskedTextGenerator(sample_tokens.numpy())

#cria o modelo bert pra o mlm e resume com summary()
bert_masked_model = create_masked_language_bert_model()
bert_masked_model.summary()

Model: "masked_bert_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ word_embedding      │ (None, 256, 128)  │  3,840,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, 256, 128)  │     32,768 │ word_embedding[0… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256, 128)  │          0 │ word_embedding[0… │
│                     │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_multihea… │ (None, 256, 128)  │     66,048 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0],        │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_att_drop… │ (None, 256, 128)  │          0 │ encoder_0_multih… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256, 128)  │          0 │ add[0][0],        │
│                     │                   │            │ encoder_0_att_dr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_att_laye… │ (None, 256, 128)  │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_ffn       │ (None, 256, 128)  │     33,024 │ encoder_0_att_la… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_ffn_drop… │ (None, 256, 128)  │          0 │ encoder_0_ffn[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 256, 128)  │          0 │ encoder_0_att_la… │
│                     │                   │            │ encoder_0_ffn_dr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_ffn_laye… │ (None, 256, 128)  │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlm_cls (Dense)     │ (None, 256,       │  3,870,000 │ encoder_0_ffn_la… │
│                     │ 30000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,842,352 (29.92 MB)

 Trainable params: 7,842,352 (29.92 MB)

 Non-trainable params: 0 (0.00 B)